# 02 — Recife vs Salvador: Análise de SLA e Cobertura de Demanda

Este notebook compara **Recife** e **Salvador** como candidatos a CD, usando:
- **Matriz OD para capitais do NE** (`data/processed/osrm/od_capitais_recife_salvador.csv`)
- **Matriz OD para Top N municípios por demanda** (`data/processed/osrm/od_municipios_topN_recife_salvador.csv`)
- **Resumo de SLA ponderado (Top N)** (`data/processed/osrm/sla_ponderado_topN_summary.csv`)
- (Opcional) **Score de consumo municipal** (`data/processed/ibge/consumo_municipal_NE_2021.csv`) para alguns cortes.

> Dica: Execute todas as células em ordem. As figuras serão salvas em `reports/figures/` para uso no README/deck.


In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

BASE = Path.cwd().resolve().parent
P_CAPITAIS = BASE / "data" / "processed" / "osrm" / "od_capitais_recife_salvador.csv"
P_MUNI_OD  = BASE / "data" / "processed" / "osrm" / "od_municipios_topN_recife_salvador.csv"
P_SUMMARY  = BASE / "data" / "processed" / "osrm" / "sla_ponderado_topN_summary.csv"
P_SCORE    = BASE / "data" / "processed" / "ibge" / "consumo_municipal_NE_2021.csv"
FIG_DIR    = BASE / "reports" / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

df_cap = pd.read_csv(P_CAPITAIS)
df_muni = pd.read_csv(P_MUNI_OD)
df_sum = pd.read_csv(P_SUMMARY)
df_score = pd.read_csv(P_SCORE) if P_SCORE.exists() else None

len(df_cap), len(df_muni), len(df_sum), (None if df_score is None else len(df_score))


(18, 500, 2, 1794)

## 1) Capitais do NE — comparação direta (horas)


In [3]:
# Tabela larga: linhas = capitais destino; colunas = dur_h_Recife-PE / dur_h_Salvador-BA
tab_cap = df_cap.pivot(index="destino", columns="origem", values="dur_h").copy()

# Guardar nomes padrões
col_recife = [c for c in tab_cap.columns if "Recife" in c][0]
col_salv = [c for c in tab_cap.columns if "Salvador" in c][0]

tab_cap["delta_h_Salvador_menos_Recife"] = tab_cap[col_salv] - tab_cap[col_recife]
tab_cap_sorted = tab_cap.sort_values("delta_h_Salvador_menos_Recife")

tab_cap_sorted.round(2)


origem,Recife-PE,Salvador-BA,delta_h_Salvador_menos_Recife
destino,,,
Salvador-BA,10.79,0.00,-10.79
Aracaju-SE,6.76,4.45,-2.31
Teresina-PI,13.95,15.62,1.66
São Luís-MA,20.49,22.15,1.66
Maceió-AL,3.58,8.06,4.49
Fortaleza-CE,11.41,18.00,6.59
Natal-RN,4.03,14.93,10.90
João Pessoa-PB,1.76,12.65,10.90
Recife-PE,0.00,10.99,10.99


In [4]:
# Barras: diferença (Salvador - Recife), negativo = Recife mais rápido
ax = tab_cap_sorted["delta_h_Salvador_menos_Recife"].plot(kind="barh", figsize=(8,6))
ax.set_xlabel("Diferença de tempo (h) — Salvador - Recife")
ax.set_ylabel("Capital destino")
ax.set_title("Capitais NE: quem chega mais rápido? (negativo favorece Recife)")
fig_path = FIG_DIR / "capitais_delta_recife_vs_salvador.png"
plt.tight_layout(); plt.savefig(fig_path, dpi=160); plt.close()
fig_path


WindowsPath('C:/Users/Juan/magalu-cd-location/reports/figures/capitais_delta_recife_vs_salvador.png')

### Observações rápidas
- Valores **negativos** favorecem **Recife** (mais rápido).
- Valores **positivos** favorecem **Salvador**.
- Use esta figura no deck para mostrar **vantagem relativa** por capital.


## 2) Top N municípios — SLA ponderado por demanda

In [5]:
# Resumo calculado no script: média ponderada e p50/p80/p90
df_sum_display = df_sum.copy().sort_values("tempo_medio_ponderado_h")
df_sum_display


,origem,N,tempo_medio_ponderado_h,p50_h,p80_h,p90_h
0,Recife-PE,500,8.992497,10.402056,14.012472,18.495889
1,Salvador-BA,500,11.435143,11.307556,17.883222,18.254167


In [6]:
# Barras: tempo médio ponderado (h)
ax = df_sum_display.plot(x="origem", y="tempo_medio_ponderado_h", kind="bar", legend=False)
ax.set_xlabel("Origem (CD)")
ax.set_ylabel("Tempo médio ponderado (h)")
ax.set_title("Top N municípios: tempo médio ponderado por demanda")
fig_path = FIG_DIR / "topN_media_ponderada.png"
plt.tight_layout(); plt.savefig(fig_path, dpi=160); plt.close()
fig_path


WindowsPath('C:/Users/Juan/magalu-cd-location/reports/figures/topN_media_ponderada.png')

In [7]:
# CDF ponderada (curva de atendimento): % da demanda atendida até t horas
def weighted_cdf(vals, weights, grid_hours=None):
    s = pd.DataFrame({"v": vals, "w": weights}).dropna()
    if s.empty or s["w"].sum() <= 0:
        return pd.DataFrame({"h": [], "cdf": []})
    s = s.sort_values("v")
    s["cw"] = s["w"].cumsum() / s["w"].sum()
    if grid_hours is None:
        grid_hours = np.linspace(s["v"].min(), s["v"].max(), 50)
    out = []
    for h in grid_hours:
        out.append({"h": h, "cdf": float(s.loc[s["v"] <= h, "w"].sum() / s["w"].sum())})
    return pd.DataFrame(out)

# monta duas CDFs (Recife e Salvador)
vals_r = df_muni.get("dur_h_Recife-PE").values
vals_s = df_muni.get("dur_h_Salvador-BA").values
if "w_norm" in df_muni.columns:
    weights = df_muni["w_norm"].values
else:
    weights = (df_muni["demand_weight"].values / df_muni["demand_weight"].sum())

grid = np.linspace(min(np.nanmin(vals_r), np.nanmin(vals_s)),
                   max(np.nanmax(vals_r), np.nanmax(vals_s)), 80)
cdf_r = weighted_cdf(vals_r, weights, grid)
cdf_s = weighted_cdf(vals_s, weights, grid)

# Plot
plt.figure(figsize=(8,6))
plt.plot(cdf_r["h"], cdf_r["cdf"], label="Recife-PE")
plt.plot(cdf_s["h"], cdf_s["cdf"], label="Salvador-BA")
plt.xlabel("Tempo de viagem (h)")
plt.ylabel("Demanda acumulada atendida")
plt.title("CDF ponderada — % demanda atendida até t horas (Top N municípios)")
plt.legend()
fig_path = FIG_DIR / "cdf_ponderada_topN.png"
plt.tight_layout(); plt.savefig(fig_path, dpi=160); plt.close()
fig_path


WindowsPath('C:/Users/Juan/magalu-cd-location/reports/figures/cdf_ponderada_topN.png')

### Métricas de SLA (limiares)
Abaixo calculamos, para cada origem, **% da demanda (Top N)** atendida em **≤ 12h, 24h, 36h**.


In [8]:
def pct_within(vals, weights, thr):
    s = pd.DataFrame({"v": vals, "w": weights}).dropna()
    if s.empty or s["w"].sum() <= 0:
        return np.nan
    return float(s.loc[s["v"] <= thr, "w"].sum() / s["w"].sum())

thr_list = [12, 24, 36]
rows = []
for lbl, col in [("Recife-PE", "dur_h_Recife-PE"), ("Salvador-BA", "dur_h_Salvador-BA")]:
    vals = df_muni[col].values
    ws = df_muni["w_norm"].values if "w_norm" in df_muni.columns else (df_muni["demand_weight"].values / df_muni["demand_weight"].sum())
    row = {"origem": lbl}
    for t in thr_list:
        row[f"pct_<=_{t}h"] = pct_within(vals, ws, t)
    rows.append(row)

df_sla = pd.DataFrame(rows)
df_sla


,origem,pct_<=_12h,pct_<=_24h,pct_<=_36h
0,Recife-PE,0.762491,0.992044,1.0
1,Salvador-BA,0.534256,0.992648,1.0


In [9]:
# Barras separadas para 12/24/36h (salva figuras)
for t in [12, 24, 36]:
    ax = df_sla.plot(x="origem", y=f"pct_<=_{t}h", kind="bar", legend=False)
    ax.set_ylim(0,1)
    ax.set_xlabel("Origem (CD)")
    ax.set_ylabel(f"% demanda <= {t}h")
    ax.set_title(f"Cobertura de SLA (Top N): % demanda atendida em ≤ {t}h")
    fig_path = FIG_DIR / f"sla_pct_le_{t}h.png"
    plt.tight_layout(); plt.savefig(fig_path, dpi=160); plt.close()
fig_path


WindowsPath('C:/Users/Juan/magalu-cd-location/reports/figures/sla_pct_le_36h.png')

## 3) Corte por UF (entender trade-offs estaduais)


In [10]:
# Se o arquivo od_municipios_topN tiver 'sigla', dá para fazer cortes por UF
if "sigla" in df_muni.columns:
    g = (df_muni.groupby(["sigla"], as_index=False)
                  .agg(peso=("w_norm","sum"),
                       media_h_recife=("dur_h_Recife-PE","mean"),
                       media_h_salvador=("dur_h_Salvador-BA","mean"))
                  .sort_values("peso", ascending=False))
    g.head(12)
else:
    print("Coluna 'sigla' não encontrada em od_municipios_topN_recife_salvador.csv; pule esta análise.")


## 4) Sumário executivo automatizado

Gera um parágrafo com os **principais números** (média ponderada, p80/p90, e coberturas de 12/24/36h) para Recife e Salvador.


In [11]:
def fmt_pct(x): 
    return f"{100*x:.1f}%" if pd.notna(x) else "n/d"

df_sum_display = df_sum.copy().sort_values("tempo_medio_ponderado_h")
rec = df_sum_display[df_sum_display["origem"].str.contains("Recife")].iloc[0].to_dict()
sal = df_sum_display[df_sum_display["origem"].str.contains("Salvador")].iloc[0].to_dict()

sla_map = df_sla.set_index("origem").to_dict(orient="index")

summary_text = f'''
**Tempo médio ponderado (Top N):**
- Recife: {rec["tempo_medio_ponderado_h"]:.2f} h  |  P50 {rec["p50_h"]:.1f} h  ·  P80 {rec["p80_h"]:.1f} h  ·  P90 {rec["p90_h"]:.1f} h
- Salvador: {sal["tempo_medio_ponderado_h"]:.2f} h  |  P50 {sal["p50_h"]:.1f} h  ·  P80 {sal["p80_h"]:.1f} h  ·  P90 {sal["p90_h"]:.1f} h

**Cobertura de SLA (Top N):**
- Recife: ≤12h {fmt_pct(sla_map["Recife-PE"]["pct_<=_12h"])}, ≤24h {fmt_pct(sla_map["Recife-PE"]["pct_<=_24h"])}, ≤36h {fmt_pct(sla_map["Recife-PE"]["pct_<=_36h"]) }
- Salvador: ≤12h {fmt_pct(sla_map["Salvador-BA"]["pct_<=_12h"])}, ≤24h {fmt_pct(sla_map["Salvador-BA"]["pct_<=_24h"])}, ≤36h {fmt_pct(sla_map["Salvador-BA"]["pct_<=_36h"]) }
'''
print(summary_text)



**Tempo médio ponderado (Top N):**
- Recife: 8.99 h  |  P50 10.4 h  ·  P80 14.0 h  ·  P90 18.5 h
- Salvador: 11.44 h  |  P50 11.3 h  ·  P80 17.9 h  ·  P90 18.3 h

**Cobertura de SLA (Top N):**
- Recife: ≤12h 76.2%, ≤24h 99.2%, ≤36h 100.0%
- Salvador: ≤12h 53.4%, ≤24h 99.3%, ≤36h 100.0%



## 5) Próximos passos (opcional)
- Rodar matrizes com **N maior** (Top 1.000–2.000) e verificar estabilidade dos resultados.
- Calibrar perfis (velocidades) do OSRM (`car.lua`) para refletir **cenários conservadores** (chuva/estradas).
- Cruzar com **custos logísticos** e **custos imobiliários** por cidade para compor o **TCO** do CD.
- Repetir as métricas com **janelas temporais** (pico/vale) se você tiver dados de trânsito.
